url = "http://api.openweathermap.org/data/2.5/forecast?id=2747373&appid="

https://pypi.org/project/metno-locationforecast/#Usage

In [1]:
import requests
import logging
from urllib.parse import quote
import constants
from dictor import dictor

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
logging.root.setLevel('DEBUG')
logging.debug('foo')

DEBUG:root:foo


In [4]:
# wrap everything in trys
def get_coord(place=None):
    osm_endpoint = constants.osm_endpoint
    osm_query = constants.osm_query
    place_quote = quote(place)
    url = f'{osm_endpoint}{place_quote}{osm_query}'
    result = requests.get(url)
    lat = dictor(result.json()[0], 'lat')
    lon = dictor(result.json()[0], 'lon')
    display_name = dictor(result.json()[0], 'display_name')
    print(f'{display_name}: {float(lat):.2f}, {float(lon):.2f}')
    return(lat, lon)

In [5]:
coord = get_coord('Golden Colorado USA')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): nominatim.openstreetmap.org:443
DEBUG:urllib3.connectionpool:https://nominatim.openstreetmap.org:443 "GET /search/Golden%20Colorado%20USA?format=json&addressdetails=0&limit=0 HTTP/1.1" 200 None


Golden, Jefferson County, Colorado, United States of America: 39.76, -105.22


In [44]:
def update_function(self):
    # build out the constants file with a reasonable (~80 value) set of data
    is_updated = False
    data = {}
    priority = 2**15
    
    failure = (is_updated, data, priority)
    
    try:
        forecast = requests.get(f"{constants.yr_endpoint}lat={self.config['lat']}&lon={self.config['lon']}")
    except RequestException as e:
        logging.warning(e)
        return failure
        
    
    if forecast.status_code == 200:
        data['updated_at'] = dictor(forecast.json(), 'properties.meta.updated_at')
        for index, each in enumerate(dictor(forecast.json(), 'properties.timeseries')):
            data[f'{index:03}_time'] = dictor(each, 'time')
            data[f'{index:03}_symbol_code'] = f'./images/png/{dictor(each, "data.next_1_hours.summary.symbol_code")}.png'
            data[f'{index:03}_precipitation_amount'] = dictor(each, "data.next_1_hours.details.precipitation_amount")
            for datum, value in dictor(each, 'data.instant.details').items():
                data[f'{index:03d}_{datum}'] = value
        priority = self.max_priority
        is_updated = True
    else:
        return failure
    
    return is_updated, data, priority

In [45]:
update_function(self)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.met.no:443
DEBUG:urllib3.connectionpool:https://api.met.no:443 "GET /weatherapi/locationforecast/2.0/complete.json?lat=39.755543&lon=-105.2210997 HTTP/1.1" 200 3804


(True,
 {'updated_at': '2020-10-31T13:27:08Z',
  '000_time': '2020-10-31T15:00:00Z',
  '000_symbol_code': './images/png/clearsky_day.png',
  '000_precipitation_amount': 0.0,
  '000_air_pressure_at_sea_level': 1017.3,
  '000_air_temperature': 6.6,
  '000_cloud_area_fraction': 0.0,
  '000_cloud_area_fraction_high': 0.0,
  '000_cloud_area_fraction_low': 0.0,
  '000_cloud_area_fraction_medium': 0.0,
  '000_dew_point_temperature': -3.0,
  '000_fog_area_fraction': 0.0,
  '000_relative_humidity': 51.3,
  '000_ultraviolet_index_clear_sky': 0.8,
  '000_wind_from_direction': 278.4,
  '000_wind_speed': 3.2,
  '001_time': '2020-10-31T16:00:00Z',
  '001_symbol_code': './images/png/clearsky_day.png',
  '001_precipitation_amount': 0.0,
  '001_air_pressure_at_sea_level': 1018.1,
  '001_air_temperature': 12.7,
  '001_cloud_area_fraction': 1.6,
  '001_cloud_area_fraction_high': 0.0,
  '001_cloud_area_fraction_low': 0.0,
  '001_cloud_area_fraction_medium': 1.6,
  '001_dew_point_temperature': -3.5,
  '001

In [11]:
self = SelfDummy()
self.config = {'lat': coord[0], 'lon': coord[1],
               'forecasts': [0, 6, 12, 24]}

In [34]:
r = requests.get(f"{constants.yr_endpoint}lat={self.config['lat']}&lon={self.config['lon']}")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.met.no:443
DEBUG:urllib3.connectionpool:https://api.met.no:443 "GET /weatherapi/locationforecast/2.0/complete.json?lat=39.755543&lon=-105.2210997 HTTP/1.1" 200 3804


In [128]:
r.status_code

200

In [47]:
meta = dictor(r.json(), 'properties.meta')

In [48]:
meta

{'updated_at': '2020-10-31T13:27:08Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'air_temperature_max': 'celsius',
  'air_temperature_min': 'celsius',
  'cloud_area_fraction': '%',
  'cloud_area_fraction_high': '%',
  'cloud_area_fraction_low': '%',
  'cloud_area_fraction_medium': '%',
  'dew_point_temperature': 'celsius',
  'fog_area_fraction': '%',
  'precipitation_amount': 'mm',
  'relative_humidity': '%',
  'ultraviolet_index_clear_sky': '1',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s'}}

In [42]:
timeseries = dictor(r.json(), 'properties.timeseries')

In [43]:
timeseries[0]

{'time': '2020-10-31T15:00:00Z',
 'data': {'instant': {'details': {'air_pressure_at_sea_level': 1017.3,
    'air_temperature': 6.6,
    'cloud_area_fraction': 0.0,
    'cloud_area_fraction_high': 0.0,
    'cloud_area_fraction_low': 0.0,
    'cloud_area_fraction_medium': 0.0,
    'dew_point_temperature': -3.0,
    'fog_area_fraction': 0.0,
    'relative_humidity': 51.3,
    'ultraviolet_index_clear_sky': 0.8,
    'wind_from_direction': 278.4,
    'wind_speed': 3.2}},
  'next_12_hours': {'summary': {'symbol_code': 'clearsky_day'}},
  'next_1_hours': {'summary': {'symbol_code': 'clearsky_day'},
   'details': {'precipitation_amount': 0.0}},
  'next_6_hours': {'summary': {'symbol_code': 'clearsky_day'},
   'details': {'air_temperature_max': 15.5,
    'air_temperature_min': 11.3,
    'precipitation_amount': 0.0}}}}

In [49]:
# def flatten_json(y):
#     out = {}

#     def flatten(x, name=''):
#         if type(x) is dict:
#             for a in x:
#                 flatten(x[a], name + a + '_')
#         elif type(x) is list:
#             i = 0
#             for a in x:
#                 flatten(a, name + str(i) + '_')
#                 i += 1
#         else:
#             out[name[:-1]] = x

#     flatten(y)
#     return out


In [58]:
def flatten_json(y):
    out = {}
    
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], f"{name}{a}_")
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, f"{name}{i:03}_")
                i += 1
        else:
            out[name[:-1]] = (x, None)

    flatten(y)
    return out


In [93]:
def flatten_json(y):
    out = {}
    
    t = {'air_temperature': 'celcius',
         'precipitation_amount': 'mm'}
    
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], f"{name}{a}_")
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, f"{name}{i:03}_")
                i += 1
        else:
            out[name[:-1]] = [x]

    flatten(y)
    return out


In [130]:
abreviations = {'celsius': 'C',
                'farenheit': 'F',
                None: ''
                }

In [125]:
f_meta = flatten_json(meta['units'])

In [126]:
for meta_k in f_meta:
    if f_meta[meta_k][0] in abreviations.keys():
        append_val = abreviations[f_meta[meta_k][0]]
    else:
        append_val = f_meta[meta_k][0]
    f_meta[meta_k].append(append_val)

In [127]:
f_meta

{'air_pressure_at_sea_level': ['hPa', 'hPa'],
 'air_temperature': ['celsius', 'C'],
 'air_temperature_max': ['celsius', 'C'],
 'air_temperature_min': ['celsius', 'C'],
 'cloud_area_fraction': ['%', '%'],
 'cloud_area_fraction_high': ['%', '%'],
 'cloud_area_fraction_low': ['%', '%'],
 'cloud_area_fraction_medium': ['%', '%'],
 'dew_point_temperature': ['celsius', 'C'],
 'fog_area_fraction': ['%', '%'],
 'precipitation_amount': ['mm', 'mm'],
 'relative_humidity': ['%', '%'],
 'ultraviolet_index_clear_sky': ['1', '1'],
 'wind_from_direction': ['degrees', 'degrees'],
 'wind_speed': ['m/s', 'm/s']}

In [89]:
f_timeseries = flatten_json(timeseries)

In [90]:
f_timeseries

{'000_time': ['2020-10-31T15:00:00Z', None],
 '000_data_instant_details_air_pressure_at_sea_level': [1017.3, None],
 '000_data_instant_details_air_temperature': [6.6, None],
 '000_data_instant_details_cloud_area_fraction': [0.0, None],
 '000_data_instant_details_cloud_area_fraction_high': [0.0, None],
 '000_data_instant_details_cloud_area_fraction_low': [0.0, None],
 '000_data_instant_details_cloud_area_fraction_medium': [0.0, None],
 '000_data_instant_details_dew_point_temperature': [-3.0, None],
 '000_data_instant_details_fog_area_fraction': [0.0, None],
 '000_data_instant_details_relative_humidity': [51.3, None],
 '000_data_instant_details_ultraviolet_index_clear_sky': [0.8, None],
 '000_data_instant_details_wind_from_direction': [278.4, None],
 '000_data_instant_details_wind_speed': [3.2, None],
 '000_data_next_12_hours_summary_symbol_code': ['clearsky_day', None],
 '000_data_next_1_hours_summary_symbol_code': ['clearsky_day', None],
 '000_data_next_1_hours_details_precipitation_am

In [92]:
for k in f_timeseries.keys():
    for meta_k in f_meta.keys():
        if meta_k in k:
            f_timeseries[k][1] = f_meta[meta_k][0]
f_timeseries

{'000_time': ['2020-10-31T15:00:00Z', None],
 '000_data_instant_details_air_pressure_at_sea_level': [1017.3, 'hPa'],
 '000_data_instant_details_air_temperature': [6.6, 'celsius'],
 '000_data_instant_details_cloud_area_fraction': [0.0, '%'],
 '000_data_instant_details_cloud_area_fraction_high': [0.0, '%'],
 '000_data_instant_details_cloud_area_fraction_low': [0.0, '%'],
 '000_data_instant_details_cloud_area_fraction_medium': [0.0, '%'],
 '000_data_instant_details_dew_point_temperature': [-3.0, 'celsius'],
 '000_data_instant_details_fog_area_fraction': [0.0, '%'],
 '000_data_instant_details_relative_humidity': [51.3, '%'],
 '000_data_instant_details_ultraviolet_index_clear_sky': [0.8, '1'],
 '000_data_instant_details_wind_from_direction': [278.4, 'degrees'],
 '000_data_instant_details_wind_speed': [3.2, 'm/s'],
 '000_data_next_12_hours_summary_symbol_code': ['clearsky_day', None],
 '000_data_next_1_hours_summary_symbol_code': ['clearsky_day', None],
 '000_data_next_1_hours_details_precip

In [10]:
data = {}
for index, each in enumerate(p):
    data[f'{index:03}_time'] = dictor(each, 'time')
    data[f'{index:03}_symbol_code'] = f'./images/png/{dictor(each, "data.next_1_hours.summary.symbol_code")}.png'
    for datum, value in dictor(each, 'data.instant.details').items():
        data[f'{index:03d}_{datum}'] = value
print(data)

NameError: name 'p' is not defined

In [ ]:
details = {
'air_pressure_at_sea_level': ('{}_{}', convert),
'air_temperature': ('{}_{}', convert),
'cloud_area_fraction': ('{}_{}', convert),
'cloud_area_fraction_high': ('{}_{}', convert),
'cloud_area_fraction_low': ('{}_{}', convert),
'cloud_area_fraction_medium': ('{}_{}', convert),
'dew_point_temperature': ('{}_{}', convert),
'fog_area_fraction': ('{}_{}', convert),
'relative_humidity': ('{}_{}', convert),
'ultraviolet_index_clear_sky': ('{}_{}', convert),
'wind_from_direction': ('{}_{}', convert),
'wind_speed': ('{}_{}', convert),
}

In [96]:
def convert(v, u_in, u_out):
    units = {
        'c': { 'c': v , 'f': v*9/5+32, 'k': v + 273.15},
        'f': { 'c': (v-32)*5/9, 'f': v, 'k': (v-32)*5/9 + 273.15},
        'k': { 'c': v-273.15, 'f': (v - 273.15) * 9/5 + 32, 'k': v},
        'm/s': {'m/s': v, 'k/h': v*3.6, 'm/h': v*2.237, 'knt': v*1.944},
        'k/h': {'m/s': v/3.6, 'k/h': v, 'm/h': v/1.609, 'knt': v/1.852},
        'm/h': {'m/s': v/2.237, 'k/h': v*1.609, 'm/h': v, 'knt': v*1.151},
        'knt': {'m/s': v/1.151, 'k/h': v*1.852, 'm/h': v/1.151, 'knt': v}
    }
    try:
        ret_val = units[u_in][u_out]
    except KeyError:
        ret_val = 'undefined'
    return ret_val
    

In [98]:
convert(35, 'm/s', 'knt')

68.03999999999999

In [ ]:
# move into constants
keys = {'time': 'time',
        'details': 'data.instant.details',
        'next_1_hours': 'data.next_1_hours',
        'next_6_hours': 'data.next_6_hours',
        'next_12_hours': 'data.next_12_hours'}

forecast_data = []
for forecast in forecasts_json:
    each_forecast = {}
    for key, dictor_key in keys.items():
        each_forecast[key] = dictor(forecast, dictor_key)
#     each_forecast = {}
#     each_forecast['time'] = dictor(forecast, 'time')
#     each_forecast['data'] = dictor(forecast, 'data.instant.details')
#     each_forecast['next_1_hours'] = dictor(forecast, 'data.next_1_hours')
#     each_forecast['next_6_hours'] = dictor(forecast, 'data.next_6_hours')
#     each_forecast['next_12_hours'] = dictor(forecast, 'data.next_12_hours')
    forecast_data.append(each_forecast)

In [ ]:
from PIL import Image
from pathlib import Path

In [ ]:
for i in g.glob('*.png'):
    img = Image.open(i)
    n_img = Image.new("RGBA", img.size, "WHITE")
    n_img.paste(img, (0, 0), img)
    n_img.save(f'{i.stem}.png')

In [ ]:
json_extract(r.json(), 'next_12_hours')

In [9]:
from SelfDummy import SelfDummy

In [ ]:
!ln ../../library/SelfDummy.py ./
